---
## Task 2 — Creating a timetable using a greedy algorithm (set cover)

**Goal:** Given a set of subjects and a pool of teachers, choose the smallest set of teachers that cover all subjects.  
**Selection rule:** At each greedy step choose the teacher who can teach the largest number of *currently uncovered* subjects; if multiple teachers tie, choose the **youngest** teacher.


In [2]:
# Task 2 implementation: Teacher class and create_schedule

from dataclasses import dataclass, field
from typing import Set, List, Optional

@dataclass
class Teacher:
    first_name: str
    last_name: str
    age: int
    email: str
    can_teach_subjects: Set[str]
    # assigned_subjects will be set by create_schedule
    assigned_subjects: Set[str] = field(default_factory=set)

def create_schedule(subjects: Set[str], teachers: List[Teacher]) -> Optional[List[Teacher]]:
    """
    Greedy algorithm for set cover:
    - subjects: set of subjects to cover
    - teachers: list of Teacher objects
    Returns list of selected Teacher objects with assigned_subjects set,
    or None if it's impossible to cover all subjects.
    Tie-breaker: when multiple teachers cover same number of remaining subjects,
    choose the youngest teacher.
    """
    uncovered = set(subjects)
    # Make a shallow copy of teachers to avoid modifying original objects outside
    teachers_copy = [Teacher(t.first_name, t.last_name, t.age, t.email, set(t.can_teach_subjects), set()) for t in teachers]
    selected_teachers = []

    while uncovered:
        # For each teacher compute how many uncovered subjects they can teach
        candidates = []
        for t in teachers_copy:
            if t.can_teach_subjects:
                covers = t.can_teach_subjects & uncovered
                if covers:
                    candidates.append((len(covers), -t.age, t, covers))
        if not candidates:
            # No teacher can cover any remaining subject -> impossible
            return None
        # Select candidate with max number of covers, tie-break by youngest (-age)
        candidates.sort(reverse=True)  # highest len, then highest -age (i.e., youngest)
        best_count, neg_age, best_teacher, best_covers = candidates[0]
        # assign those subjects to this teacher
        best_teacher.assigned_subjects.update(best_covers)
        # remove covered subjects from uncovered
        uncovered -= best_covers
        # add teacher to result and remove them from future consideration
        selected_teachers.append(best_teacher)
        # ensure the teacher won't be selected again by clearing their cover set
        best_teacher.can_teach_subjects = set()

    return selected_teachers


In [3]:
# Example usage with the teachers and subjects provided in the assignment

subjects = {'Mathematics', 'Physics', 'Chemistry', 'Informatics', 'Biology'}

teachers = [
    Teacher('Alexander', 'Evans', 45, 'a.evans@example.com', {'Mathematics', 'Physics'}),
    Teacher('Maria', 'Peterson', 38, 'm.peterson@example.com', {'Chemistry'}),
    Teacher('George', 'Collins', 50, 'g.collins@example.com', {'Informatics', 'Mathematics'}),  # 'Computer Science' -> use 'Informatics' to match subjects
    Teacher('Natalie', 'Stevens', 29, 'n.stevens@example.com', {'Biology', 'Chemistry'}),
    Teacher('Daniel', 'Brown', 35, 'd.brown@example.com', {'Physics', 'Informatics'}),
    Teacher('Helen', 'Grant', 42, 'h.grant@example.com', {'Biology'}),
]

schedule = create_schedule(subjects, teachers)

if schedule is None:
    print("It is impossible to cover all subjects with the available teachers.")
else:
    print("Class Schedule:")
    for t in schedule:
        subjects_list = ', '.join(sorted(t.assigned_subjects))
        print(f"{t.first_name} {t.last_name}, {t.age} years old, email: {t.email}")
        print(f"   Teaches subjects: {subjects_list}\n")


Class Schedule:
Natalie Stevens, 29 years old, email: n.stevens@example.com
   Teaches subjects: Biology, Chemistry

Daniel Brown, 35 years old, email: d.brown@example.com
   Teaches subjects: Informatics, Physics

Alexander Evans, 45 years old, email: a.evans@example.com
   Teaches subjects: Mathematics



### Explanation of Task 2 design & correctness

**Greedy choice:** Choose the teacher who covers the largest number of still-uncovered subjects.  
**Tie-breaker:** If several teachers cover the same number of uncovered subjects, pick the youngest teacher (`-age` used in sorting). This matches the assignment requirement.

**Why greedy works reasonably well here:** The set-cover problem is NP-hard in general; the greedy heuristic is a standard approximation algorithm with good practical performance. For small fixed inputs (as in the assignment), the greedy approach often yields the optimal or near-optimal solution.

**Edge cases handled**
- If some subject cannot be covered by any teacher, the function returns `None`, and we print the requested message.
- Each selected teacher receives `assigned_subjects` only from the set of uncovered subjects they contribute at the moment they are chosen. This ensures that all assigned_subjects are disjoint across teachers in the resulting schedule (which is fine for the assignment).
